###Hidden Markov Models

Implementation of example from wikipedia.

Doctor records symptoms of patient each day. Symptom can be 0, 1 or 2. Doctor needs to decide if the patient is healthy (0) or has a fever (1). Transition probabilties described by a. Emission probabilties by e and the initial probability distribution by p. The observed symptoms are in the list, y.

In [86]:
import numpy as np
a = np.array([[.7,.3],[.4,.6]])
#a = np.array([[.9,.1],[.1,.9]])
e = np.array([[.5,.4,.1],[.1,.3,.6]])
p = np.array([.6,.4])
ys = [0,1,2]
path = np.zeros((len(ys),2))
for i,y in enumerate(ys):
    tempP = np.zeros((2,2))
    if (i==0):
        transition_probs = np.array([[1.,0.],[0.,1.]]) #no transitions the first iteration
    else:
        transition_probs = a
    for s in range(2):
        for t in range(2):
            tempP[s,t] = transition_probs[s,t]*e[t,y]*p[s]
    p = np.max(tempP,axis=0)
    idx = np.argmax(p)    
    p = p / np.sum(p)
    path[i,:] = np.argmax(tempP,axis=0)
    #print tempP
node = np.argmax(p)
route = []
for step in path[::-1]:
    route.insert(0,int(node))
    node = step[node]
print route

[0, 0, 1]


In [243]:
import numpy as np
def HMM_viterbi(a,e,p,ys):
    #a=transition matrix, e=emission matrix, p=initial state, ys=observations
    Nstates = a.shape[0]
    path = np.zeros((len(ys),Nstates))
    for i,y in enumerate(ys):
        tempP = np.zeros((Nstates,Nstates))
        if (i==0):
            transition_probs = np.eye(Nstates) #no transitions the first iteration
        else:
            transition_probs = a
        for s in range(Nstates):
            for t in range(Nstates):
                tempP[s,t] = transition_probs[s,t]*e[t,y]*p[s]
        p = np.max(tempP,axis=0)
        idx = np.argmax(p)    
        p = p / np.sum(p)
        path[i,:] = np.argmax(tempP,axis=0)
        #print tempP
    node = np.argmax(p)
    route = []
    for step in path[::-1]:
        route.insert(0,int(node))
        node = step[node]
    return route
    
a = np.array([[.7,.3],[.4,.6]])
e = np.array([[.5,.4,.1],[.1,.3,.6]])
p = np.array([.6,.4])
ys = [0,1,2]
HMM_viterbi(a,e,p,ys)

[0, 0, 1]

In [39]:
import urllib2, json
from threading import Thread
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

class Test():
    def geonames_api_query(self,place):
        name = place['word']
        from urllib import urlencode
        name = name.encode('utf8')
        #url = 'http://api.geonames.org/searchJSON?name_equals=%s&featureCode=PPLA2&featureCode=PPLX&featureCode=PPLA3&featureCode=PPLA4&featureCode=PPLA&featureCode=PPLC&maxRows=10&username=lionfish' % name
        url = 'http://api.geonames.org/searchJSON?name_equals=%s&featureCode=ADM1&featureCode=PCLI&featureCode=PPLX&featureCode=PPLA2&featureCode=ADM2&featureCode=ADM3&featureCode=ADM4&featureCode=PPLA3&featureCode=PPLA4&featureCode=PPLA&featureCode=PPLC&maxRows=10&username=lionfish' % name
        #http://api.geonames.org/searchJSON?name_equals=burma&maxRows=10&username=lionfish
        raw_json = urllib2.urlopen(url).readline() 
        data = json.loads(raw_json)
        if 'geonames' not in data:
            place['found'] = False
            return
        
        place['found'] = False
        lastpop = -1
        for placedata in data['geonames']:
            if placedata['name'].lower()==name.lower():
                if placedata['population']>lastpop: #go with the biggest
                    lastpop = placedata['population']
                    place['lat'] = placedata['lat']
                    place['lng'] = placedata['lng']
                    place['raw'] = placedata                
                    place['found'] = True
                
            

    def get_places(self,data):
        s = WordNetLemmatizer()
        places = []
        for like in data['likes']['data']:
            datetime = like['created_time']
            for fullword in like['name'].split(' '):
                word = s.lemmatize(fullword.lower()) #strips -s etc from end of word
                if word in words.words():
                    continue 
                places.append({'word':word,'datetime':datetime})
       
        threads = []
        for place in places:
            t = Thread(target = self.geonames_api_query, args=(place,))
            threads.append(t)
            t.start()

        for t in threads:
            t.join()
        return places
            
t = Test()


with open('facebookdataexample.txt') as data_file:    
    data = json.load(data_file)
            
places = t.get_places(data)

In [48]:
[p['raw']['countryName'] for p in places if p['found']]

[u'United Kingdom',
 u'United Kingdom',
 u'Uganda',
 u'United Kingdom',
 u'Uganda',
 u'United Kingdom',
 u'United Kingdom',
 u'South Africa',
 u'United Kingdom',
 u'Malawi',
 u'United Kingdom',
 u'United Kingdom',
 u'United Kingdom',
 u'United Kingdom']

In [50]:
import shapefile
from shapely.geometry import Point
from shapely.geometry import Polygon

def getcountry(points):
    ppoints = []
    res = []
    for pnti, pnt in enumerate(points):
        ppoints.append(Point(pnt[0],pnt[1])) #convert all points to Point() class
        res.append(None) #we prefill our results list with a not-found "None"
    recs = []
    sf = shapefile.Reader('countryboundaries/world-country-boundaries.shp')
    #sf = shapefile.Reader('cb/ne_110m_admin_0_countries.shp')
    for i in range(sf.numRecords):
        shp = sf.shape(i)
        rec = sf.record(i)
        recs.append(rec)
        bbox= shp.bbox
     #   print bbox
        #loop through all the points, checking if they're in each country.
        for pnti, p in enumerate(ppoints):
            if (rec[1]=='USA'):
                if res[pnti]!=None:
                    continue #TODO: Hack to handle bug where some points in the UK are set to the USA
            if p.within(Polygon(shp.points)): 
                print "Within"
                countryname = rec[3]
                regionname = rec[57]
                res[pnti] = i,rec,points[pnti]
    return res, recs
p = [(-1.,53.)]
res,rec= getcountry(p)

print regionname
print countryname


Within
Europe & Central Asia
United Kingdom


##### points = []
for p in places:
    if p['found']:
        points.append((float(p['lng']),float(p['lat']),p))

In [353]:
res, recs = getcountry(points)

In [326]:
#probability of staying in the same country for each like
pStay = 0.9 #0.8
#the following probabilities get spread across...
 #...the countries in the region
pMoveInRegion = 0.4 #0.1
 #...all countries
pMove = 0.1
pEmitRightCountry = 4./14. #only calculated approximately; 4/14 from my own facebook.

N = len(recs)
a = np.zeros((N,N))
for i,recI in enumerate(recs):
    for j,recJ in enumerate(recs):
        if recI[2]==recJ[2]: #if in same region of world
            a[i,j] = 1
a = pMoveInRegion*(a/np.tile(np.sum(a,axis=0),(N,1)))
a[range(N),range(N)] = np.ones(N)*pStay
a = a + pMove/N

#calculation was a bit rough, so normalising
a = a/np.tile(np.sum(a,axis=0),(N,1))
#probability of 'emitting' that you are in your current country or not
e = (1-pEmitRightCountry) * np.ones((N,N))/(N)
e = e + np.eye(N)*pEmitRightCountry
p = np.ones(N)/N

In [70]:
res.insert(2,res[2])
res.insert(2,res[2])
res.insert(2,res[2])


#probability of staying in the same country for each like
pStay = 0.9 #0.8
#the following probabilities get spread across...
 #...the countries in the region
pMoveInRegion = 0.4 #0.1
 #...all countries
pMove = 0.1
pEmitRightCountry = 4./14. #only calculated approximately; 4/14 from my own facebook.

N = len(recs)
a = np.zeros((N,N))
for i,recI in enumerate(recs):
    for j,recJ in enumerate(recs):
        if recI[2]==recJ[2]: #if in same region of world
            a[i,j] = 1
a = pMoveInRegion*(a/np.tile(np.sum(a,axis=0),(N,1)))
a[range(N),range(N)] = np.ones(N)*pStay
a = a + pMove/N

#calculation was a bit rough, so normalising
a = a/np.tile(np.sum(a,axis=0),(N,1))
#probability of 'emitting' that you are in your current country or not
e = (1-pEmitRightCountry) * np.ones((N,N))/(N)
e = e + np.eye(N)*pEmitRightCountry
p = np.ones(N)/N

guess = HMM_viterbi(a,e,p,[r[0] for r in res])
print [(r[0],r[2][2]['datetime'][0:4]) for r in res]
print guess

IndexError: list index out of range

In [69]:
import collections
counts = collections.Counter(guess)
c = counts.items()
print c
newlist = sorted(c, key=lambda k: -k[1]) 
print newlist

[((2010, 'Malawi'), 1), ((2009, 'UK'), 1), ((2012, 'UK'), 1)]


TypeError: bad operand type for unary -: 'tuple'

In [67]:
import time
countries = []
for item in newlist:
    idx = (len(guess) - 1) - guess[::-1].index(item[0])
    datetime = res[idx][2][2]['datetime']
    t = time.strptime(datetime, '%Y-%m-%dT%H:%M:%S+0000')
    countries.append((recs[item[0]],t))
print [c[0][0] for c in countries]

IndexError: list index out of range

In [452]:
import sys
if len(countries)==1:
    con = countries[0]
    print "Since at least %d you've lived in %s" % (con[1][0],con[0][0])
else:
    con = countries[0]
    print "Since at least %d you've mainly lived in %s" % (con[1][0],con[0][0]),
    con = countries[1]
    sys.stdout.write(" but have also lived in %s" % con[0][0])
    for i,con in enumerate(countries[2:-1]):
        sys.stdout.write(", %s" % (con[0][0]))
    if len(countries)>2:
        sys.stdout.write(" and %s" % countries[-1][0][0])
sys.stdout.flush()

Since at least 2015 you've mainly lived in United Kingdom but have also lived in United States


In [54]:
sf = shapefile.Reader('countryboundaries/world-country-boundaries.shp')
#sf = shapefile.Reader('cb/ne_110m_admin_0_countries.shp')
places = []
for i in range(sf.numRecords):
    places.append(sf.record(i))

In [64]:
[p[0] for p in places].index()

172

['Brazil', 'BRA', 'Latin America']

In [72]:
msg = "sdfgsdg"
msg += "dg"
msg

'sdfgsdgdg'